# Multilingual Embedding-based Machine Translation

Работа выполнена в рамках курса естественно-научного содержания (ЕНС) _**"Элементы машинного обучения"**_  
Преподаватель: Шокуров Антон Вячеславович

Автор: Хритова Екатерина Андреевна  
Группа: **510**

**Содержание**
> [Общие сведения](#section-1)   
>> [Данные](#subsection-11)  
>> [Мера качества](#subsection-12)  
>> [Embeding](#subsection-13)  

> [Mодель №0. Идеальный перевод](#section-2)   
> [Mодель №1](#section-3)  
> [Модель №2](#section-4)   
> [Модель №3](#section-5)   

> [Результаты](#section-6) 

> [Задача классификации сообщений](#section-7)  
>>[Данные](#subsection-71)  
>>[Модель классификации](#subsection-72)  
>>[Переводчик](#subsection-73)  
>>[Перевод + классификация](#subsection-74)  

Этот блокнот основан на статьях:  
[1] Samuel L. Smith, David H. P. Turban, Steven Hamblin & Nils Y. Hammerla, [Offline bilingual word vectors, orthogonal transformations and
the inverted softmax](https://openreview.net/pdf?id=r1Aab85gg)   
[2] A. Conneau*, G. Lample*, L. Denoyer, MA. Ranzato, H. Jégou, [Word Translation Without Parallel Data](https://arxiv.org/pdf/1710.04087.pdf)  
[3] Armand Joulin Piotr Bojanowski Tomas Mikolov, [Loss in Translation: Learning Bilingual Word Mapping with a Retrieval Criterion](https://arxiv.org/pdf/1804.07745.pdf)

Рассмотрим задачу машинного перевода. Пусть задано представление слов (как векторов $x \in \mathbb{R}^d$). Для решения задачи будем выравнивать векторы из двух языков в едином векторном пространстве с помощью некоторого отображения $W$. Ниже представлена схема выравнимания для векторного представления английских слов $X$ и испанских слов $Y$.

![embedding_mapping.png](https://github.com/yandexdataschool/nlp_course/raw/master/resources/embedding_mapping.png)

Рассмотрим следующие модели:
1. Использующая линейное отображение между непрерывными представлениями слов.
2. Использующая линейное ортогональное отображение между непрерывными представлениями слов.
3. Использующая линейное ортогональное отображение между непрерывными представлениями слов, при построении которого используется корректирующая метрика.


<a name="section-1"></a> 
# Общие сведения <a name="subsection-11"></a> 

Для исследования выберем два родственных славянских языка: **украинский** и **русский**. 

<a name="subsection-13"></a> 
## Embeding

Для embeding-а возьмем уже готовые представления:
* [cc.uk.300.vec.zip](https://yadi.sk/d/9CAeNsJiInoyUA)
* [cc.ru.300.vec.zip](https://yadi.sk/d/3yG0-M4M8fypeQ)

Посмотрим на слова, наиболее близкие к слову _"август"_ и _"серпень"_ (перевод на украинский язык)

In [1]:
import pandas as pd
from gensim.models import KeyedVectors

In [2]:
# EMBEDING
uk_emb = KeyedVectors.load_word2vec_format("cc.uk.300.vec")
ru_emb = KeyedVectors.load_word2vec_format("cc.ru.300.vec")

In [4]:
ruemb_to_ru = ru_emb.most_similar([ru_emb["август"]])
#ukemb_to_ro = uk_emb.most_similar([ru_emb["август"]])
ukemb_to_uk = uk_emb.most_similar([uk_emb["серпень"]])
ruemb_to_uk = ru_emb.most_similar([uk_emb["серпень"]])

print('Наиболее близкие к слову "август" для ru_emb:')
for k in range(10):
    print("{:10} {}".format(ruemb_to_ru[k][0], round(ruemb_to_ru[k][1], 3)))
    
print('\nНаиболее близкие к слову "серпень" для uk_emb:')
for k in range(10):
    print("{:10} {}".format(ukemb_to_uk[k][0], round(ukemb_to_uk[k][1], 3)))

print('\nНаиболее близкие к слову "серпень" для ru_emb:')
for k in range(10):
    print("{:10} {}".format(ruemb_to_uk[k][0], round(ruemb_to_uk[k][1], 3)))

Наиболее близкие к слову "август" для ru_emb:
август     1.0
июль       0.938
сентябрь   0.924
июнь       0.922
октябрь    0.91
ноябрь     0.893
апрель     0.873
декабрь    0.865
март       0.855
февраль    0.84

Наиболее близкие к слову "серпень" для uk_emb:
серпень    1.0
липень     0.91
вересень   0.902
червень    0.899
жовтень    0.881
листопад   0.879
квітень    0.859
грудень    0.859
травень    0.841
лютий      0.826

Наиболее близкие к слову "серпень" для ru_emb:
Недопустимость 0.244
конструктивность 0.233
офор       0.233
deteydlya  0.23
пресечении 0.226
одностороннего 0.226
подход     0.223
иболее     0.22
2015Александр 0.219
конструктивен 0.218


## Данные

В качестве исходного датасета используется файл, содержащий украинские и русские слова, разделенные табуляцией. Все буквы в словах записаны в нижнем регистре, слова не содаржат знаков препинания, цифр и пр.  
Пример нескольких таких пар:

|Ukrainian  | Russian   |
|-----------|-----------|
|автомобіль | автомобиль|
|автомобіль | вагон     |
|агов       | эй        |
|аґрус      |крыжовник  |
|адже       |ведь
|...        | ...       |

Загрузка данных

In [5]:
import numpy as np

In [6]:
def load_word_pairs(filename):
    uk_ru_pairs = []
    uk_vectors = []
    ru_vectors = []
    with open(filename, "r") as inpf:
        for line in inpf:
            uk, ru = line.rstrip().split("\t")
            if uk not in uk_emb or ru not in ru_emb:
                continue
            uk_ru_pairs.append((uk, ru))
            uk_vectors.append(uk_emb[uk])
            ru_vectors.append(ru_emb[ru])
    return uk_ru_pairs, np.array(uk_vectors), np.array(ru_vectors)

In [7]:
uk_ru_train, X_train, Y_train = load_word_pairs("ukr_rus_train_fixed.txt")
uk_ru_test, X_test, Y_test = load_word_pairs("ukr_rus_test_fixed.txt")

<a name="subsection-12"></a>  
## Мера качества

В качестве меры качества будем ипользовать долю "близко угаданных" слов с точностью top-1, top-5 и top-10:

Для каждого преобразованного украинского представления ищется его $N$ ближайших соседей в пространстве представления русских слов (т.е. слово "переводится" с украинского на русский и для полученного "перевода" ищется $N$ наиболее близких по смыслу русских слов). Далее находится частота "близко угаданных" слов, т.е. тех, у которых хотя бы одна точка в найденной окрестности совпадает с истинным переводом.

Аргументы функции:
* Массив _pairs_ содержит пары слов из словаря (т.е. слово и его точный перевод)
* Массив _mapped_vectors_ - это массив построенных приближений/переводов дли всех украинских слов из массива _pairs_
* topn - точность (количество элементов в окрестности)

In [8]:
def precision(pairs, mapped_vectors, topn=1):
    assert len(pairs) == len(mapped_vectors)
    num_matches = 0.
    for i, (_, ru) in enumerate(pairs):
        neighbours = ru_emb.most_similar(positive=[mapped_vectors[i]], topn=topn)
        for neigh, _ in neighbours:
                if neigh == ru:
                    num_matches += 1.
    precision_val = num_matches / len(pairs)
    print(precision_val)
    return precision_val

<a name="section-2"></a>
# Модель №0 (Идеальный перевод)

In [11]:
def ideal_translation(pairs, X):
    assert len(pairs) == len(X)
    true_trans = []
    for (_, ru) in pairs:
        true_trans.append(ru_emb[ru])
    return np.array(true_trans)

При идеальном переводе, какую бы окрестность мы не взяли, она всегда будет содержать истинный ответ. Поэтому _precision_ для истинного перевода всегда равен 1.0

In [18]:
print("Ideal translation for train set")
assert precision(uk_ru_train, ideal_translation(uk_ru_train,X_train), topn=1) == 1.0
assert precision(uk_ru_train, ideal_translation(uk_ru_train,X_train), topn=5) == 1.0
assert precision(uk_ru_train, ideal_translation(uk_ru_train,X_train), topn=10) == 1.0

print("\nIdeal translation for test set")
assert precision(uk_ru_test, ideal_translation(uk_ru_test,X_test), topn=1) == 1.0
assert precision(uk_ru_test, ideal_translation(uk_ru_test,X_test), topn=5) == 1.0
assert precision(uk_ru_test, ideal_translation(uk_ru_test,X_test), topn=10) == 1.0

Ideal translation for train set
1.0
1.0
1.0

Ideal translation for test set
1.0
1.0
1.0


<a name="section-3"></a>
# Модель №1

Пусть $x_i \in \mathbb{R}^d$ — представление слова $i$ на исходном языке, а $y_i \in \mathbb{R}^d$ — представление его перевода. Наша цель — узнать такое линейное преобразование $W$, которое минимизирует евклидово расстояние между $Wx_i$ и $y_i$ для некоторого подмножества представлений слов. Таким образом, мы можем сформулировать так называемую проблему Прокруста:

$$W^*= \arg\min_W \sum_{i=1}^n||Wx_i - y_i||_2$$

Заметим, что $W^*= \arg\min_W \sum_{i=1}^n||Wx_i - y_i||_2$ выглядит как простая множественная линейная регрессия (без перехвата). Итак, давайте искать ее именно в таком виде.

In [198]:
from sklearn.linear_model import LinearRegression

mapping = LinearRegression(fit_intercept=False)
mapping.fit(X_train, Y_train)

#Y_pred = mapping.predict(X_test)

LinearRegression(copy_X=True, fit_intercept=False, n_jobs=None, normalize=False)

Посмотрим на соседей вектора слова _"сiчень"_ после линейного преобразования.

In [9]:
august = mapping.predict(uk_emb["серпень"].reshape(1, -1))
ruemb_to_uk = ru_emb.most_similar(august)

print('Наиболее близкие к слову "серпень" для ru_emb:')
for k in range(10):
    print("{:10} {}".format(ruemb_to_uk[k][0], round(ruemb_to_uk[k][1], 3)))

Наиболее близкие к слову "серпень" для ru_emb:
апрель     0.854
июнь       0.841
март       0.84
сентябрь   0.836
февраль    0.833
октябрь    0.831
ноябрь     0.828
июль       0.824
август     0.812
декабрь    0.804


Таким образом, окрестность этого вложения состоит из разных месяцев, но правильный вариант находится на девятом месте.

Т.е., если мы вычислим меру качества модели для выборки, состоящей только из слова _август_ (и его перевода), то оценка с точность top-5 даст 0.0, а оценка с точностью top-9 и выше даст 1.0

In [10]:
assert precision([("серпень", "август")], august, topn=5) == 0.0
assert precision([("серпень", "август")], august, topn=9) == 1.0
assert precision([("серпень", "август")], august, topn=10) == 1.0

0.0
1.0
1.0


Вычислим теперь меру качества построенной модели для тестовой выборки.

In [11]:
precision_top1 = precision(uk_ru_test, mapping.predict(X_test), 1)
precision_top5 = precision(uk_ru_test, mapping.predict(X_test), 5)
precision_top10 = precision(uk_ru_test, mapping.predict(X_test), 10)

0.6424870466321243
0.8082901554404145
0.8497409326424871


<a name="section-4"></a>
# Модель №2

Можно показать (см. оригинальную статью), что самосогласованное (каждая точка на гладкой кривой или поверхности является средним значением всех точек, которые проецируются на нее ортогонально) линейное отображение между семантическими пространствами должно быть ортогональным.
Тогда можно искать отображение $W$ как ортогональное:

$$W^*= \arg\min_W ||WX - Y||_2 ,\\ W^TW = I, \\I \text{- identity matrix}$$

Оптимальное ортогональное преобразование можно найти, используя разложение по сингулярным значениям. Оказывается, оптимальное преобразование $W^*$ можно выразить через компоненты SVD:

$$X^TY=U\Sigma V^T\text{, сингулярное разложение}$$
$$W^*=UV^T$$

In [181]:
#singular value decompostion

def learn_transform(X_train, Y_train):
    U, S, V = np.linalg.svd(np.matmul(X_train.T, Y_train))
    return np.matmul(U,V)

In [182]:
W2 = learn_transform(X_train, Y_train)

In [183]:
prescision_top1 = precision(uk_ru_test, np.matmul(X_test, W2), 1)
prescision_top5 = precision(uk_ru_test, np.matmul(X_test, W2), 5)
prescision_top10 = precision(uk_ru_test, np.matmul(X_test, W2), 10)

0.655440414507772
0.8238341968911918
0.8523316062176166


<a name="section-5"></a>
# Модель №3

Можно показать, что построенные модели страдают от так называемой "hubness problem": некоторые векторы слов имеют тенденцию быть ближайшими соседями аномально большого количества других слов. Одним из решений этой проблеммы является использование при выводе корректирующей метрики. В [[3]](https://arxiv.org/pdf/1804.07745.pdf) в качестве такой метрики выбирается CSLS-критерий:
$$CSLS(x,y) = -2 cos(x,y) + \frac{1}{k}\sum_{y' \in \mathcal{N}_Y(x)} cos(x, y') + \frac{1}{k}\sum_{x' \in \mathcal{N}_X(y)} cos(x', y) $$

Здесь $\mathcal{N}_Y(x)$ - окрестность точки $x$ в пространстве $Y$ (k ближайших соседей) и $cos(x,y)$ - cosine similarity (косинусное сходство)

В терминах нашей задачи:
$$cos(Wx_i, y_j) = x_i^TW^Ty_j$$
В то же время
$$||Wx_i - y_j||_2^2 = 2 - 2x_i^TW^Ty_j$$

Другими словами, $k$ ближайщих соседей можно искать как $k$ элементов, для которых значение $x^TW^Ty$ будет максимальным.

Отображение $W$, как и в предыдущей модели, будем искать как ортогональное $W \in \mathcal{O}_d$.

Задача оптимизации (_Relaxed CSLS loss_) в этом случае записывается как
$$W^* = \underset{W \in \mathcal{O}_d}{argmin}(\frac{1}{n}\sum_{i=1}^{n} -2x_i^TW_i^Ty_i + \frac{1}{k}\sum_{y_j\in\mathcal{N}_Y(Wx_i)}x_i^TW^Ty_j +  \frac{1}{k}\sum_{Wx_j\in\mathcal{N}_X(y_i)}x_j^TW^Ty_i)$$

In [187]:
'''
RCSLS-frmula
Arguments:
    * X - vectors of the source language
    * Y - vectors of the target language
    * W - mapping
    * k - number of points in the neighborhood
'''

def RCSLS(X, Y, W, k = 10):
    n = X.shape[0]
    X_trans = np.matmul(X, W.T)
    # first term
    f1 = 2*np.sum(X_trans * Y)
    df1 = 2 * np.matmul(Y.T, X)
    #second term
    f2, df2 = knn_term(np.matmul(X_trans, Y.T), X, Y, k)
    #third term
    f3, df3 = knn_term(np.matmul(np.matmul(X, W.T), Y.T).T, Y, X, k)
    
    f = -f1 + f2 + f3
    df = -df1 + df2 + df3.T
    
    return f/n, df/n

In [188]:
'''
The second and third terms in the RCSLS-formula
Arguments:
    * term_val - x^T*W^T*y (k nearest neighbors can be searched as k elements for which this value will be maximum)
    * space_pnt - source/target language
    * space_neigh - target/source language
    * k - number of points in the neighborhood
'''

def knn_term(term_val, space_pnt, space_neigh, k):
    # find k nearest neighbors
    neigh_pid = np.argpartition(term_val, -k, axis=1)[:, -k:]
    shape = neigh_pid.shape
    neigh = space_neigh[neigh_pid.flatten(), :]
    neigh = neigh.reshape(shape[0], shape[1], space_neigh.shape[1])
    
    f = np.sum(term_val[np.arange(term_val.shape[0])[:, None], neigh_pid])
    df = np.matmul(neigh.sum(1).T, space_pnt)
    
    return f/k, df/k

Зададим некоторые параметры

In [189]:
# number of subsample elements
subsetsize = 1000
# number of iterations
niter = 25
# number of nearest neighbors
k = 20
# learning rate
lr = 1.0

In [190]:
#singular value decompostion
W3 = learn_transform(X_train, Y_train)

X = X_train.copy()
Y = Y_train.copy()

f_old = 0
W_old = []

for it in range(0, niter + 1):
    if lr < 1e-4:
        break
    ids = np.random.choice(X.shape[0], size=subsetsize, replace=False)
    f, df = RCSLS(X[ids, :], Y[ids, :], W3, k)      
    W3 -= lr * df 
    f_old, W_old = f, W3

In [39]:
prescision_top1 = precision(uk_ru_test, np.matmul(X_test, W3), 1)
prescision_top5 = precision(uk_ru_test, np.matmul(X_test, W3), 5)
prescision_top10 = precision(uk_ru_test, np.matmul(X_test, W3), 10)

0.6606217616580311
0.8341968911917098
0.8549222797927462


<a name = "section-6"></a>
# Результаты

Оценки, полученные для каждой модели, приведены в следующей таблице:

|      | Модель №1 | Модель №2 | Модель №3 |
|------|-----------|-----------|-----------|
|top-1 | 0.6425    | 0.6554    | 0.6606    |
|top-5 | 0.8083    | 0.8238    | 0.8342    |
|top-10| 0.8497    | 0.8523    | 0.8549    |


Как и ожидалось, модель №1 является худшей, а модель №3 показывает лучший результат.

<a name="section-7"></a>
# Задача классификации сообщений

In [319]:
import warnings
warnings.filterwarnings('ignore')

<a name="subsection-71"></a>
## Данные

Рассмотрим задачу классификации сообщений. В качестве [данных](https://www.kaggle.com/ihelon/ukrainian-descriptions-of-words/version/3?select=index.csv) используем описание 15 различных слов 8 различными участниками (на украинском языке)

In [315]:
df = pd.read_csv("index.csv")
df

,Unnamed: 0,user,word,description
0,0,1,кінь,"Парнокопитна тварина, яка була однією з перших..."
1,1,1,зебра,"африканська тварина, зовнішній вигляд якої наг..."
2,2,1,корова,"вид парнокопитної худоби, основне використання..."
3,3,1,ведмідь,"всеїдна велика тварина, яка має дуже сильний т..."
4,4,1,олень,"парнокопитна дика тварина, яка зазвичай прожив..."
...,...,...,...,...
115,115,8,морквина,"Городня рослина, овоч з їстівним солодкуватим ..."
116,116,8,яблуко,"соковитий плід, який вживається в їжу в свіжом..."
117,117,8,банан,фрукт з шкіркою. Колір шкірки може бути: жовти...
118,118,8,картопля,"Рід овоча, рослина з їстівними бульбами, багат..."


Добавим метки классов (цифры вместо слов)

In [316]:
words = []
word_ids = []
i = 0
for word in set(df["word"]):
    words.append(word)
    word_ids.append(i)
    i = i+1
    
word_codes = {words[i]: word_ids[i] for i in range(len(words))}

class_id = []
for word in df["word"]:
    class_id.append(word_codes[word])
    
df['class'] = class_id

df

,Unnamed: 0,user,word,description,class
0,0,1,кінь,"Парнокопитна тварина, яка була однією з перших...",13
1,1,1,зебра,"африканська тварина, зовнішній вигляд якої наг...",2
2,2,1,корова,"вид парнокопитної худоби, основне використання...",0
3,3,1,ведмідь,"всеїдна велика тварина, яка має дуже сильний т...",3
4,4,1,олень,"парнокопитна дика тварина, яка зазвичай прожив...",5
...,...,...,...,...,...
115,115,8,морквина,"Городня рослина, овоч з їстівним солодкуватим ...",10
116,116,8,яблуко,"соковитий плід, який вживається в їжу в свіжом...",1
117,117,8,банан,фрукт з шкіркою. Колір шкірки може бути: жовти...,6
118,118,8,картопля,"Рід овоча, рослина з їстівними бульбами, багат...",7


В качестве обучающей выборки выберем ответы случайных 5 участников опроса, а в качестве тестовой - ответы оставшихся 3 участников

In [380]:
import random

tus = random.sample(range(1,9), 3)
print("Test dataset from users: ", tus)

df_train = df[(df['user'] != tus[0]) & (df['user'] != tus[1]) & (df['user'] != tus[2])]
df_test = df[(df['user'] == tus[0]) | (df['user'] == tus[1]) | (df['user'] == tus[2])]

Test dataset from users:  [2, 6, 4]


<a name="subsection-72"></a>
## Модель классификации

Для решения задачи классификации, представим тексты в виде разреженной матрицы матрицы токенов, находящихся в тексте, при помощи метода _CountVectorizer_ библиотеки _sklearn_. Для классификации полученных преставлений используем (многоклассовый) наивный баясовский классификатор.

In [368]:
from sklearn.feature_extraction.text import CountVectorizer 

#from sklearn.linear_model import LogisticRegression
#from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB

from sklearn.metrics import balanced_accuracy_score

In [376]:
# for the mathematical description of sentences
# ignore words that appear in texts less than 2 times
# ignore words with a frequency higher than 0.8
vectorizer =  CountVectorizer(
              lowercase=True, ngram_range=(2,3), analyzer='char',
              preprocessor=preprocessor, min_df=2, max_df=0.8)

# for classification
#classificator = LogisticRegression(solver = 'liblinear')
#classificator = DecisionTreeClassifier(random_state = 42)
classificator = MultinomialNB()

In [381]:
# Train

text_train = df_train['description']
text_train_vect = vectorizer.fit_transform(text_train)

y_train = df_train['class']

classificator.fit(text_train_vect, y_train)
y_train_pred = classificator.predict(text_train_vect)

balanced_accuracy_score(y_train_pred, y_train)

1.0

In [382]:
# Test

text_test = df_test['description']
text_test_vect = vectorizer.transform(text_test)

y_test = df_test['class']

y_test_pred = classificator.predict(text_test_vect)

balanced_accuracy_score(y_test_pred, y_test)

0.851111111111111

<a name="subsection-73"></a>
## Переводчик

"Пословный" переводчик предложений: каждое слово заменяется на его "перевод"

Аргументы функции:
* sentence - предложение на украинском языке
* translator - перефодчик (матрица W для моделей 2 и 3 или обученная линейная регрессия для модели 1)
* translate_w_matrix - маркер, значение _True_ для моделей 2,3 и _False_ для модели 1

In [235]:
import string
from copy import deepcopy


def translate(sentence, translator, translate_w_matrix = True):
    ru_sentence = deepcopy(sentence)
    no_punct = ''.join(c for c in sentence if c not in string.punctuation)
    words = no_punct.split()
    for i, word in enumerate(words):
        if word.lower() in uk_emb.vocab:
            is_capital = word[0].isupper()
            if(translate_w_matrix):
                translation = ru_emb.most_similar([np.matmul(uk_emb[word.lower()], translator)])[0][0]
            else:
                translation = ru_emb.most_similar(translator.predict(uk_emb[word.lower()].reshape(1, -1)))[0][0]
            if is_capital:
                translation = translation[0].upper() + translation[1:]
            ru_sentence = ru_sentence.replace(word, translation)
    return ru_sentence

In [236]:
sentence = df_train['description'][36]
print(sentence)
print(translate(sentence, mapping, translate_w_matrix = False))
print(translate(sentence, W2))
print(translate(sentence, W3))

домашня птиця, яка майже не літає, кудахкає, використовується людьми в основному для їжі
домашняя птица, она почти не летает, кудахкає, используется людьми во основоному для пищи
домашняя птица, она почти не летает, кудахкає, используется людьми во основоному для пищи
домашняя птица, она почти не летает, кудахкає, используется людьми во основоному для еды


<a name="subsection-74"></a>
## Перевод + классификация

In [260]:
def precision_class(y_pred, y_true):
    assert len(y_pred) == len(y_true)
    n = len(y_pred)
    d = y_pred - y_true
    return len(np.argwhere(d==0))/n

In [246]:
def translate_col(model, df_col):
    assert model in [1,2,3]
    translation = []
    if model == 1:
        for sentence in df_col:
            translation.append(translate(sentence, mapping, translate_w_matrix = False))
    elif model == 2:
        for sentence in df_col:
            translation.append(translate(sentence, W2))
    elif model == 3:
        for sentence in df_col:
            translation.append(translate(sentence, W3))
    return  np.array(translation)     

In [383]:
mod1_transl = translate_col(1, df['description'])
mod2_transl = translate_col(2, df['description'])
mod3_transl = translate_col(3, df['description'])

df['model1'] = mod1_transl
df['model2'] = mod2_transl
df['model3'] = mod3_transl

df_train = df[(df['user'] != tus[0]) & (df['user'] != tus[1]) & (df['user'] != tus[2])]
df_test = df[(df['user'] == tus[0]) | (df['user'] == tus[1]) | (df['user'] == tus[2])]

In [386]:
df_test[:5]

,Unnamed: 0,user,word,description,class,model1,model2,model3
15,15,2,кінь,"копитна тварина, на якій людина здатна швидко ...",13,"копитпо животное, по которой людипо здатпо быс...","копитпо животное, по которой людипо здатпо быс...","копитпо животное, по которой людипо здатпо быс..."
16,16,2,зебра,"тварина, яка схожа на коня, має забарвлення в ...",2,"живоотное, опо похожа по конь, иметь цвоет во ...","живоотное, опо похожа по коня, имеет окраску в...","живоотное, опо похожа по коня, имеет окраску в..."
17,17,2,корова,"копитна тварина з вим'ям, мичить і дає молоко",0,"копитна животное со вим'ям, мичить и дает молоко","копитна животное со вим'ям, мичить и дает молоко","копитна животное со вим'ям, мичить и дает молоко"
18,18,2,ведмідь,"лісова всеїдна тварина, яка полюбляє мед, може...",3,"лесная зверек животное, она любит мёд, может л...","лесная стайная животное, она любит мёд, может ...","лесная стайная животное, она любит мёд, может ..."
19,19,2,олень,лісова тварина з рогами,5,лесная животное со рогами,лесная животное со рогами,лесная животное со рогами


In [388]:
y_train = np.array(df_train['class'])
y_test  = np.array(df_test['class'])

for model in ["model1","model2","model3"]:
    # Train
    text_train_trans = df_train[model]
    text_train_vect = vectorizer.fit_transform(text_train_trans)
    
    # Test
    text_test_trans = df_test[model]
    text_test_vect = vectorizer.transform(text_test)

    classificator.fit(text_train_vect, y_train)
    y_train_pred = classificator.predict(text_train_vect)
    y_test_pred = classificator.predict(text_test_vect)

    train_score = balanced_accuracy_score(y_train_pred, y_train)
    test_score = balanced_accuracy_score(y_test_pred, y_test)
    #train_score = precision_class(y_train_pred, y_train)
    #test_score = precision_class(y_test_pred, y_test)
    print("model № {} : \n\tTrain score: {} \n\tTest  score: {}".format(model, train_score, test_score))

model № model1 : 
	Train score: 1.0 
	Test  score: 0.8285714285714285
model № model2 : 
	Train score: 1.0 
	Test  score: 0.8844444444444444
model № model3 : 
	Train score: 1.0 
	Test  score: 0.8744444444444444


Для данного разбиения модель №3 показала результат лучше, чем модель №2. Однако, оба они сильно уступают первой модели. Попробуем снизить зависимость результата от разбиения выборки. Для этого выполним обучение и прогноз 50 раз и в качестве ответа представим среднее значение полученных результатов.

In [393]:
df[:5]

,Unnamed: 0,user,word,description,class,model1,model2,model3
0,0,1,кінь,"Парнокопитна тварина, яка була однією з перших...",13,"Парнокопитна животное, она была одним со первы...","Парнокопитна животное, она была одним со первы...","Парнокопитна животное, она была одним со первы..."
1,1,1,зебра,"африканська тварина, зовнішній вигляд якої наг...",2,"африканская животное, внешний вид которой напо...","африканская животное, внешний вид которой напо...","африканская животное, внешний вид которой напо..."
2,2,1,корова,"вид парнокопитної худоби, основне використання...",0,"вид парнокопитної скот, основное использование...","вид парнокопитної скота, основное использовани...","вид парнокопитної скота, основное использовани..."
3,3,1,ведмідь,"всеїдна велика тварина, яка має дуже сильний т...",3,"зверек большая животное, она иметь очень сильн...","стайная большая животное, она имеет очень силь...","стайная большая животное, она имеет очень силь..."
4,4,1,олень,"парнокопитна дика тварина, яка зазвичай прожив...",5,"парнокопитна дикий животное, она обычно прожив...","парнокопитна дикая животное, она обычно прожив...","парнокопитна дикая животное, она обычно прожив..."


In [407]:
for i in range(50):
    tus = random.sample(range(1,9), 3)

    df_train = df[(df['user'] != tus[0]) & (df['user'] != tus[1]) & (df['user'] != tus[2])]
    df_test = df[(df['user'] == tus[0]) | (df['user'] == tus[1]) | (df['user'] == tus[2])]
    
    y_train = np.array(df_train['class'])
    y_test  = np.array(df_test['class'])
    
    m1_res_train = []
    m2_res_train = []
    m3_res_train = []
    m1_res_test = []
    m2_res_test = []
    m3_res_test = []
    
    for model in ["model1","model2","model3"]:
        # Train
        text_train_trans = df_train[model]
        text_train_vect = vectorizer.fit_transform(text_train_trans)
    
        # Test
        text_test_trans = df_test[model]
        text_test_vect = vectorizer.transform(text_test)

        classificator.fit(text_train_vect, y_train)
        y_train_pred = classificator.predict(text_train_vect)
        y_test_pred = classificator.predict(text_test_vect)

        train_score = balanced_accuracy_score(y_train_pred, y_train)
        test_score = balanced_accuracy_score(y_test_pred, y_test)
        
        if model == "model1":
            m1_res_train.append(train_score)
            m1_res_test.append(test_score)
        if model == "model2":
            m2_res_train.append(train_score)
            m2_res_test.append(test_score)
        if model == "model3":
            m3_res_train.append(train_score)
            m3_res_test.append(test_score)
            

print("model № 1 : \n\tTrain score: {} \n\tTest  score: {}".format(np.mean(m1_res_train), np.mean(m1_res_test)))
print("model № 2 : \n\tTrain score: {} \n\tTest  score: {}".format(np.mean(m2_res_train), np.mean(m2_res_test)))
print("model № 3 : \n\tTrain score: {} \n\tTest  score: {}".format(np.mean(m3_res_train), np.mean(m3_res_test)))

model № 1 : 
	Train score: 1.0 
	Test  score: 0.9055555555555554
model № 2 : 
	Train score: 1.0 
	Test  score: 0.9166666666666666
model № 3 : 
	Train score: 1.0 
	Test  score: 0.94


Для средних значений лидирует модель с номером 3!